In [9]:

import os
import json
from typing import List
import awswrangler as wr
import boto3
from tenacity import *
from yarl import URL
from datetime import datetime
from utils.read_config import read_toml_config
from utils.call_get import call_get
from utils.logger import setup_applevel_logger


ROOT_DIR = "C:\\Users\\u.darhevich\\Documents\\master_project\\src"
CONFIG_PATH = os.path.abspath(os.path.join(ROOT_DIR, '..', 'config', 'config.toml'))

config = read_toml_config(CONFIG_PATH)
API_ENDPOINT = URL(config['dev']['api_endpoint'])
PATH_COIN_LIST = config['dev']['path_coin_list']
PATH_RAW_DATA = config['dev']['path_raw_data']
PATH_TO_LOGS = os.path.abspath(os.path.join(ROOT_DIR, '..', config['dev']['path_to_logs']))
TODAY = datetime.today().strftime("%Y%m%d")
AWS_PROFILE = config['dev']['aws_profile']
AWS_BUCKET = config['dev']['aws_bucket']
LAMBA_FUNC = config['dev']['lambda_func_name']

log = setup_applevel_logger(file_name = PATH_TO_LOGS + "/logging_{}".format(TODAY))

session = boto3.Session(profile_name=AWS_PROFILE)
s3_client = session.client('s3')
lambda_client = session.client('lambda')

def get_historical_data(id: str) -> List[str]:
   
    return json.loads(
            call_get(API_ENDPOINT / "coins" / id / "market_chart", {"vs_currency" : "usd", "days" : "max"})
        )

# def save_raw_json(data, coin: str):
#     with open(PATH_RAW_DATA + '/{}_historical_prices.json'.format(coin), 'w') as f:
#         json.dump(data, f)

def lambda_get_data(func_name: str, params: dict, client=None):
    
    if not client:
        client = boto3.client('lambda')
    response = client.invoke(
            FunctionName=func_name,
            InvocationType='Event',
            Payload=json.dumps(params)
        )
    return response

def download_coins(coin: str):
    params = {
            "bucket": AWS_BUCKET,
            "path_raw_data": PATH_RAW_DATA
        }
    params['coin'] = coin
    lambda_get_data(LAMBA_FUNC, params, lambda_client)
    log.info(f"{coin} - raw json file was saved")


def download_all_coins(coins):
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(download_coins, coins)

if __name__ == "__main__":
    import concurrent.futures
    import requests
    import threading
    import time

    thread_local = threading.local()

    path_list_of_coins = f"s3://{AWS_BUCKET}/{PATH_COIN_LIST}/list_of_coins.csv"
    list_of_coins = wr.s3.read_csv([path_list_of_coins], boto3_session=session, encoding='utf8')['id'].to_list()[1800:2000] #for test only
    download_all_coins(list_of_coins)
    
    




2022-04-10 13:07:36,072 - MyAwesomeApp - INFO - black-label - raw json file was saved
2022-04-10 13:07:36,084 - MyAwesomeApp - INFO - black-phoenix - raw json file was saved
2022-04-10 13:07:36,088 - MyAwesomeApp - INFO - blacknet - raw json file was saved
2022-04-10 13:07:36,097 - MyAwesomeApp - INFO - blackpearl-chain - raw json file was saved
2022-04-10 13:07:36,099 - MyAwesomeApp - INFO - black-market-gaming - raw json file was saved
2022-04-10 13:07:36,099 - MyAwesomeApp - INFO - black-kishu-inu - raw json file was saved
2022-04-10 13:07:36,111 - MyAwesomeApp - INFO - black-lemon - raw json file was saved
2022-04-10 13:07:36,150 - MyAwesomeApp - INFO - blackmoon-crypto - raw json file was saved
2022-04-10 13:07:36,718 - MyAwesomeApp - INFO - blackpool-token - raw json file was saved
2022-04-10 13:07:36,758 - MyAwesomeApp - INFO - black-shiba - raw json file was saved
2022-04-10 13:07:36,832 - MyAwesomeApp - INFO - blank - raw json file was saved
2022-04-10 13:07:36,834 - MyAwesome